In [3]:
!pip install gradio
!pip install python-docx
!pip install PyPDF2
!pip install sentence-transformers


In [4]:
import gradio as gr
import PyPDF2
import docx
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load the pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to extract text from uploaded file
def extract_text(file):
    name = file.name
    if name.endswith('.pdf'):
        reader = PyPDF2.PdfReader(file)
        return ''.join([page.extract_text() for page in reader.pages if page.extract_text()])
    elif name.endswith('.docx'):
        doc = docx.Document(file)
        return '\n'.join([para.text for para in doc.paragraphs])
    elif name.endswith('.txt'):
        return file.read().decode()
    return ""

# Function to calculate similarity scores between JD and each resume
def match_resumes(jd_file, resume_files):
    jd_text = extract_text(jd_file)
    jd_embedding = model.encode(jd_text, convert_to_tensor=True)

    scores = []
    for resume_file in resume_files:
        resume_text = extract_text(resume_file)
        candidate_name = resume_text.strip().split('\n')[0] or "Unnamed Candidate"  # First line as name

        resume_embedding = model.encode(resume_text, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(jd_embedding, resume_embedding)
        score = round(similarity.item() * 100, 2)

        scores.append({"Candidate Name": candidate_name, "Match Score (%)": score})

    # Create DataFrame and sort by score
    df = pd.DataFrame(scores)
    df = df.sort_values(by="Match Score (%)", ascending=False).reset_index(drop=True)

    # Extract top match info
    top = df.iloc[0]
    top_summary = f"""
## 🏆 Most Eligible Candidate

**{top['Candidate Name']}**
✔️ Match Score: **{top['Match Score (%)']}%**

---

## 📊 All Candidate Scores:
"""

    return df, top_summary

# Build the Gradio UI
gr.Interface(
    fn=match_resumes,
    inputs=[
        gr.File(label="📄 Upload Job Description", file_types=[".pdf", ".docx", ".txt"]),
        gr.File(label="📁 Upload Multiple Resumes", file_types=[".pdf", ".docx", ".txt"], file_count="multiple")
    ],
    outputs=[
        gr.Dataframe(label="📊 Candidate Scoreboard"),
        gr.Markdown(label="🏅 Top Match Summary")
    ],
    title="💼 AI Resume Matcher",
    description="Upload a Job Description and multiple resumes (PDF/DOCX/TXT). The app extracts real candidate names and ranks them by match score."
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://acf549f81925bc9ed4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
